In [2]:
import importlib

In [15]:
import hnsw
importlib.reload(hnsw)
from hnsw import HNSW
from hnsw import heuristic
import numpy as np
from tqdm import tqdm
from datasets import load_sift_dataset, calculate_recall

In [16]:
import merge_hnsw
importlib.reload(merge_hnsw)
from merge_hnsw import hnsw_general_merge
# from merge_hnsw import merge_naive

In [26]:
def merge_naive(hnsw_a, hnsw_b, merged_data, level, search_ef=5):
    '''
    hnsw_a    – the first hnsw graph 
    hnsw_b    – the second hnsw graph
    level     – mering level number
    search_ef – ef parameter for searching candidates in the second graph
                  
    '''
    m = hnsw_a._m0 if level == 0 else hnsw_a._m
    merged_edges = {}
    for curr_idx in tqdm(hnsw_a._graphs[level].keys()): 
        observed = hnsw_b.search(q=hnsw_a.data[curr_idx], k=m, ef=search_ef, level=level, return_observed=True) #return_observed=True
        candidates_b = observed[:m]
        # candidates_b = observed
        # == build neighborhood for curr_idx and save to externalset of edges  ==
        candidates = [ (idx_b, dist) for idx_b, dist in candidates_b] + [ (idx, dist) for idx, dist in hnsw_a._graphs[level][curr_idx]]
        # merged_edges[curr_idx] = sorted ([ (idx_b + len(kga.data), dist) for idx_b, dist in candidates_b] + [ (idx, dist) for idx, dist in kga.edges[curr_idx]], key=lambda a: a[1])[:k]
        merged_edges[curr_idx] = hnsw_a.neighborhood_construction(candidates, hnsw_a.data[curr_idx], m, hnsw_a.distance_func, merged_data)    
        # == == == == == == == == == == == == == == == == == == == == == == == ==

    for curr_idx in tqdm(hnsw_b._graphs[level].keys()): 
        observed = hnsw_a.search(q=hnsw_b.data[curr_idx], k=m, ef=search_ef, level=level, return_observed=True)
        candidates_a = observed[:m]
        # candidates_a = observed
        # == build neighborhood for curr_idx and save to externalset of edges  ==
        candidates = [(idx_a, dist) for idx_a, dist in candidates_a] + [(idx, dist) for idx, dist in hnsw_b._graphs[level][curr_idx]]
        merged_edges[curr_idx] = hnsw_b.neighborhood_construction(candidates, hnsw_b.data[curr_idx], m, hnsw_a.distance_func, merged_data)
        # == == == == == == == == == == == == == == == == == == == == == == == ==

    return merged_edges

In [27]:
distance_count = 0
def l2_distance(a, b):
    global distance_count
    distance_count+=1
    return np.linalg.norm(a - b)

In [28]:
hnsw_a = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)
hnsw_b = HNSW( distance_func=l2_distance, m=5, m0=7, ef=10, ef_construction=30,  neighborhood_construction = heuristic)

In [29]:
hnsw_a.load('save/sift1m/hnsw_a.txt')
hnsw_b.load('save/sift1m/hnsw_b.txt')

In [30]:
merged_data = hnsw_a.data.copy()
merged_data.update(hnsw_b.data)

In [31]:
merge_ef = 20
def layer_merge_naive_func(hnsw_a, hnsw_b, merged_data, level):
  return merge_naive(hnsw_a, hnsw_b, merged_data, level, search_ef=merge_ef)

In [32]:
%%time
distance_count = 0
hnsw_merged_naive_m = hnsw_general_merge(hnsw_a, hnsw_b, merged_data, layer_merge_naive_func)
print(f'Numer of distance calculated during the merge: {distance_count}') # 696869746

Merging level: 0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500000/500000 [3:39:05<00:00, 38.04it/s]


Merging level: 1


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31358/31358 [01:45<00:00, 297.23it/s]


Merging level: 2


...

Merging level: 3


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 761.68it/s]


Merging level: 4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 977.05it/s]

Numer of distance calculated during the merge: 960554605
CPU times: user 2h 6min 21s, sys: 8min 14s, total: 2h 14min 35s
Wall time: 4h 41min 32s


In [33]:
hnsw_merged_naive_m.save(f'save/sift1m/hnsw_merged_naive_m_ef{merge_ef}.txt')

In [34]:
print(f'Numer of distance calculated during the merge: {distance_count}') # 960554605

Numer of distance calculated during the merge: 960554605


In [35]:
_, test_data, groundtruth_data = load_sift_dataset(train_file = None,
                                                      test_file='../datasets/sift1m-128d/sift_query.fvecs',
                                                      groundtruth_file='../datasets/sift1m-128d/sift_groundtruth.ivecs')

In [39]:
search_ef = 32
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive_m, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:10<00:00, 141.88it/s]

32,0.8433, 604.5765


In [40]:
search_ef = 40
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive_m, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:29<00:00, 111.29it/s]

40,0.8733800000000002, 700.0272


In [41]:
search_ef = 64
distance_count = 0
recall, _ = calculate_recall(hnsw_merged_naive_m, test_data, groundtruth=groundtruth_data, k=5, ef=search_ef)
print(f'{search_ef},{recall}, {distance_count/len(test_data) }') 

Calculating recall...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [02:49<00:00, 58.96it/s]

64,0.9237400000000001, 977.8111
